# Lecture 11 - Files (https://bit.ly/39pWEle)

Today:
* Files.
  * Writing Files
  * Reading files
  * With keyword
  * For loops on file handles
  * Binary files 
  * Large worked example (Fasta file processing)
  * Directories
  * Web file retrieval



# Motivation

* Up to now we've mainly used input() to get input to the programs we've written.

* In the real world, this is not the way we generally get data into a program. Rather we read and write data to and from files.

* Files, like documents, or spread sheets or images, are just semantically organized arrays of bits. 

* They can exist on a disk, be pulled from the internet or be created transiently as we pipe inputs and outputs from our programs. 

* They can be small, and they can be very large. Typically the data associated with a scientific analysis will be large relative to the size of the program. For this reason we have to think about processing the data in stages using streaming. 

Anyway, let's dive in... (as an aside, the examples today will show larger code samples than we've seen previously - you're leveling up)

# Writing Files

Let's first look at creating a file. You'll often want to do this when writing out the results of some analysis.

In [ ]:
my_file = open("test.txt", "w") # Call to open() opens a file
# test.txt in the working directory of the Python program.
# The "w" string argument (short for "write"), tells open()
# that you intend to write to the file. 

# If test.txt does not exist before this point it is created as an empty file.

# If test.txt existed before this point its contents are now overwritten - 
# so be careful opening files for writing

# my_file (the return value from open) is a "file handle", that is you use it
# to handle the communication with the file



# The write method of the file_handle object is a lot like print(), except
# the contents of the string arguments are written to the file, instead of the
# screen, and write does not add newlines to the end of the string (so you add them
# explicitly, as in this example)
my_file.write("My first file written from Python\n") 
my_file.write("---------------------------------\n")
my_file.write("Hello, world!\n")

my_file.close() # This closes the file. You can't keep using my_file after this
# as the file connection is closed. It is important to close the file when 
# you're done to ensure the file writing is completed successfully (although Python
# will clean things up automatically on exit, you shouldn't rely on this in cases
# where you're program gets terminated prematurely)

Note: (Do matched demo from terminal to show contents of file)


**To append to the end of an existing file** 

In [ ]:
my_file = open("test.txt", "a") # The 'a' argument (short for append) opens an 
# existing file to append to it 
# If test.txt does not exist then 'a' creates the file, just like 'w'

my_file.write("Hello, again!\n") 

my_file.close() # Again, always close the file

Note: (Continue matched demo from terminal to show contents of appended to file)

# Reading Files

In [ ]:
mynewhandle = open("test.txt", "r") # This opens the file, "r" argument means "read"

# The following loop walks through
# the file line by line

while True:                            # Keep reading forever
  
    theline = mynewhandle.readline()   # Try to read next line using readline()
    
    if theline == "":              # When done, readline returns the empty string ''
      # allowing us to detect the end of the file and to leave the loop
      
        break                          #     leave the loop

    # Now process the line we've just read
    print(theline), end="")

mynewhandle.close() # This might seem less important, as you're just reading 
# from the file, however most underlying operating systems have limits on
# the number of active file handles, so it is good to cleanup. It is also 
# generally a bad idea to have multiple file handles open on one file (unless they are
# all read only file handles)

My first file written from Python

---------------------------------

Hello, world!

Hello, again!



What is nice about the above loop is that regardless of the number of files, Python only stores one line in memory, allowing it to process arbitrarily large files.

Python will complain if you try to open a file that doesn't exist...

In [ ]:
mynewhandle = open("wharrah.txt", "r")

FileNotFoundError: ignored

To read contents of file into memory in one go use read()

In [ ]:
fh = open("test.txt", "r") # I often use fh to denote a file handle, for some reason

content = fh.read() # Read the contents of the file

print(type(content)) # Yup, content is a string

print(content)

fh.close()

<class 'str'>
My first file written from Python
---------------------------------
Hello, world!
Hello, again!



readlines will read all the lines in a file:

In [ ]:
fh = open("test.txt") # I often use fh to denote a file handle, for some reason
# If you don't give the second argument it defaults to read only

content = fh.readlines() # Read the contents of the file

print(type(content)) # Yup, content is now a list of strings

print(content)

fh.close()

<class 'list'>
['My first file written from Python\n', '---------------------------------\n', 'Hello, world!\n', 'Hello, again!\n']


# The With Keyword

Good programming is often about anticipating errors and dealing with them.

The 'with' keyword is both useful short-hand and good defensive programming. We don't need to know the details (curious: https://docs.python.org/2.5/whatsnew/pep-343.html), but basically it cleans up the file handle for you

In [ ]:
with open("test.txt") as fh: # With removes the need for the "fh.close()" statement
  # and is better because fh.close() is guaranteed to be run even if there is an error
  # in processing the file
  
  assert False # This causes an error, but fh will still be closed
  
  content = fh.readlines() 
  print(content)
  
  
# For files the syntax is:
# with OPEN_EXPRESSION as FILE_HANDLE:
#   STATEMENTS THAT USE FILE_HANDLE

AssertionError: ignored

# For loop on file:

In [ ]:
# If you run a for loop on a file handle open for reading then
# it defaults to reading lines, this is compact and memory efficient (because
# we only read a line at a time from the file)

with open("test.txt") as fh:
  for line in fh:
    print(line, end="") # end="" stops print() from adding a newline to the end

My first file written from Python
---------------------------------
Hello, world!
Hello, again!


# Binary Files

All the files we've seen so far are assumed to be text files (i.e. composed of human readable stuff). However, Python can also happily process binary files, which are just files with arbitrary bits in them. 

On the left is a binary file - the bits aren't organized so that they can be decoded as text. We can't assume that the bits in a binary file are organized into bytes that represent text characters with white space, etc. We can always treat a text file as a binary file, it's just that we ignore its formatting.

On the right is a text file - the bits are formatted so that each byte or 16 bit word represents a text character.

![alt text](https://www.syncfusion.com/books/Java_Succinctly_Part_2/Images/binary-file-vs-text-file.png)


In [ ]:
# Reading and writing binary data

with open("test.txt", "rb") as f: # Here we treat test.txt as a binary file - any file
  # can be considered just a collection of bits
  
  with open("test2.txt", "wb") as g: # Note the nested with statements

    while True:
      buf = f.read(1024) # The argument to read tells it to read a set number of 
      # bytes - each byte is an 8 bit (8 0's or 1's) words into the "buf" object, 
      # which is a string

      #print(buf)
      
      if len(buf) == 0: # We're at the end
         break
          
      g.write(buf)

b'My first file written from Python\n---------------------------------\nHello, world!\nHello, again!\n'
b''


Now we can see that test2.txt is just a copy of the original file:

In [ ]:
with open("test2.txt") as fh:
  for line in fh:
    print(line, end="") # end="" stops print() from adding a newline to the end

My first file written from Python
---------------------------------
Hello, world!
Hello, again!


# File Processing Example

Here's a fairly complete worked example of doing file writing and parsing using the simplest bioinformatics format, the fasta file.

A fasta file is a file that stores nucleotide (like DNA, RNA,) and amino-acid (protein) sequences.

You should be able to understand all this code and what it is doing. If you don't, play with it until you do!




In [ ]:
"""
Example creating read and write methods for fasta files.

Fasta format is a dead simple text (ascii) file:

>HEADER_1
SEQUENCE_LINE_1
SEQUENCE_LINE_2
...
SEQUENCE_LINE_N
>HEADER_2
SEQUENCE_LINE_1
SEQUENCE_LINE_2
...
SEQUENCE_LINE_N

Where HEADER_ lines give a string descriptor of the sequence
and the SEQUENCE_LINE_ lines are concatenated together to form the actual 
amino-acid or nucleotide sequence.
"""

def writeFasta(fileHandle, header, sequence, sequenceLineWidth=100):
  """ Function writes a fasta file to the given file handle.
  """
  fileHandle.write('>' + header + "\n") # Write the header line
  
  # Now write the sequence
  for i in range(0,len(sequence),sequenceLineWidth): # Step through sequence sequenceLineWidth characters
    # at a time
    
    # Write the next sequenceLineWidth chars of the sequence
    fileHandle.write(sequence[i:i+sequenceLineWidth] + "\n") 
    
def readFasta(fileHandle):
  """ Read a single sequence and header from a fasta file.
  """
  
  # Read the header line, skipping and lines not beginning with a '>'
  while True: # Keep looping until we break
    l = fileHandle.readline()
    
    if len(l) == 0: # If we have reached the end of the file, terminate
      return None # None is a standard NULL or 0 quantity used to indicate
    # termination
    
    if l[0] == '>': # Is a valid header line
      header = l[1:-1] # Get the  header between the '>' and the '\n'
      break
   
  # Now read the sequence
  substrings = []
  while True:
    i = fh.tell() # tell() gives the index that the file handle is at in the file
    
    l = fileHandle.readline()
    
    if len(l) == 0: # We have reached the end of the file
      break
    
    if l[0] == '>': # We have encountered the start of another sequence
      fh.seek(i) # Roll back the file handle to the point before we called readline()
      # this is like "undoing" the readline() call
      break
  
    substrings.append(l[:-1]) # Add the line, minus the newline to substrings
  
  return header, "".join(substrings)
  
# A few test sequences
dna_sequences = [ 
    ("a_dummy_dna_string", "GATTACA"),
    ("gi 556503834 ref NC_000913.3 Escherichia coli str. K-12 substr. MG1655, complete genome", "AGCTTTTCATTCTGACTGCAACGGGCAATATGTCTCTGTGTGGATTAAAAAAAGAGTGTCTGATAGCAGCTTCTGAACTGGTTACCTGCCGTGAGTAAATTAAAATTTTATTGACTTAGGTCACTAAATACTTTAACCAATATAGGCATAGCGCACAGACAGATAAAAATTACAGAGTACACAACATCCATGAAACGCATTAGCACCACCATTACCACCACCATCACCATTACCACAGGTAACGGTGCGGGCTGACGCGTACAGGAAACACAGAAAAAAGCCCGCACCTGACAGTGCGGGCTTTTTTTTTCGACCAAAGGTAACGAGGTAACAACCATGCGAGTGTTGAAGTTCGGCGGTACATCAGTGGCAAATGCAGAACGTTTTCTGCGTGTTGCCGATATTCTGGAAAGCAATGCCAGGCAGGGGCAGGTGGCCACCGTCCTCTCTGCCCCCGCCAAAATCACCAACCACCTGGTGGCGATGATTGAAAAAACCATTAGCGGCCAGGATGCTTTACCCAATATCAGCGATGCCGAACGTATTTTTGCCGAACTTTTGACGGGACTCGCCGCCGCCCAGCCGGGGTTCCCGCTGGCGCAATTGAAAACTTTCGTCGATCAGGAATTTGCCCAAATAAAACATGTCCTGCATGGCATTAGTTTGTTGGGGCAGTGCCCGGATAGCATCAACGCTGCGC"),
    ("another_dummy_dna_string", "CGGCGCGCGTCTTTGCAGCGATGTCACGCGCCCGTATTTCCGTGGTGCTGATTACGCAATCATCTTCCGAATACAGCATCAGTTTCTGCGTTCCACAAAGCGACTGTGTGCGAGCTGAACGGGCAATGCAGGAAGAGTTC"),]

# Write the sequences into the file
with open("fasta.fa", "w") as fh:
  for header, sequence in dna_sequences:
    writeFasta(fh, header, sequence)
    
# Let's take a quick look at the file we wrote:
with open("fasta.fa") as fh:
  print("Printing fasta file")
  for line in fh:
    print(line, end="") # end="" stops print() from adding a newline to the end

# Now let's read them back
dna_sequences_copy = []
with open("fasta.fa") as fh:
  while True:
    i = readFasta(fh)
    if i == None:
      break
    dna_sequences_copy.append(i)
    
# We can check it all worked
assert dna_sequences == dna_sequences_copy

Printing fasta file
>a_dummy_dna_string
GATTACA
>gi 556503834 ref NC_000913.3 Escherichia coli str. K-12 substr. MG1655, complete genome
AGCTTTTCATTCTGACTGCAACGGGCAATATGTCTCTGTGTGGATTAAAAAAAGAGTGTCTGATAGCAGCTTCTGAACTGGTTACCTGCCGTGAGTAAAT
TAAAATTTTATTGACTTAGGTCACTAAATACTTTAACCAATATAGGCATAGCGCACAGACAGATAAAAATTACAGAGTACACAACATCCATGAAACGCAT
TAGCACCACCATTACCACCACCATCACCATTACCACAGGTAACGGTGCGGGCTGACGCGTACAGGAAACACAGAAAAAAGCCCGCACCTGACAGTGCGGG
CTTTTTTTTTCGACCAAAGGTAACGAGGTAACAACCATGCGAGTGTTGAAGTTCGGCGGTACATCAGTGGCAAATGCAGAACGTTTTCTGCGTGTTGCCG
ATATTCTGGAAAGCAATGCCAGGCAGGGGCAGGTGGCCACCGTCCTCTCTGCCCCCGCCAAAATCACCAACCACCTGGTGGCGATGATTGAAAAAACCAT
TAGCGGCCAGGATGCTTTACCCAATATCAGCGATGCCGAACGTATTTTTGCCGAACTTTTGACGGGACTCGCCGCCGCCCAGCCGGGGTTCCCGCTGGCG
CAATTGAAAACTTTCGTCGATCAGGAATTTGCCCAAATAAAACATGTCCTGCATGGCATTAGTTTGTTGGGGCAGTGCCCGGATAGCATCAACGCTGCGC
>another_dummy_dna_string
CGGCGCGCGTCTTTGCAGCGATGTCACGCGCCCGTATTTCCGTGGTGCTGATTACGCAATCATCTTCCGAATACAGCATCAGTTTCTGCGTTCCACAAAG
CGACTGTGTGCGAGCTGAACGGGCAATGC

# Directories

In most systems the operating has access to a file system in which files are kept. Getting into details
is beyond the scope of this course, because it requires us getting into operating system stuff - but let's cover the very basics.

For more see: https://docs.python.org/3/library/os.html and https://docs.python.org/3/library/os.path.html

By default when you open/write a file in Python uses the current working directory.

In [ ]:
import os # This module provides all sorts of useful functions 
# for working with the operating system

os.getcwd() # getcwd() tells us what working directory we're in. 
# (If you're using Google Collaboratory, this is some directory on
# some (virtual) machine in Google Cloud Platform)

'/content'

In [ ]:
os.listdir(os.getcwd()) # listdir() tells us what files are in the working directory

['.config', 'test.txt2', 'test.txt', 'test2.txt', 'fasta.fa', 'sample_data']

In [ ]:
# We can create a directory like so:

os.mkdir("new_directory")

os.listdir(os.getcwd() + "/new_directory") # Now "new_directory" is a directory in our working directory

[]

In [ ]:
# We can write a file in our new directory like so:

# Python uses strings to indicate file paths, generally we can indicate file
# paths with the / symbol, so new_directory/test.txt is a file "test.txt" in the 
# directory "new_directory"

with open("new_directory/test.txt", "w") as fh:
  for header, sequence in dna_sequences: # Write the DNA sequences into the file
    writeFasta(fh, header, sequence)
    
    
os.listdir(os.getcwd() + "/new_directory") # Now "new_directory" is a directory in our working directory

['test.txt']

It is worth mentioning how to delete a file, because your Python programs/scripts should try not make lots of messy files:

In [ ]:
os.remove("new_directory/test.txt") # This removes the file test.txt from new_directory

os.listdir(os.getcwd() + "/new_directory") # Now the directory is empty

[]

In [ ]:
# Similarly you can remove empty directories
os.rmdir("new_directory") # This will error out if the directory is not empty

os.listdir(os.getcwd()) # new_directory is gone

['.config', 'test.txt2', 'test.txt', 'test2.txt', 'fasta.fa', 'sample_data']

# Getting Data from the Internet

In [ ]:
import urllib.request

url = "https://raw.githubusercontent.com/DataBiosphere/toil/master/src/toil/test/wdl/test.csv"
local_copy = "local.txt"

urllib.request.urlretrieve(url, local_copy) # This function copies the thing the url points at into
# a local file copy

with open(local_copy) as fh: # Print the file
  for line in fh:
    print(line, end="")

1,2,3
4,5,6
7,8,9


# Challenge Example

In [ ]:
# Complete the following code.

l = list(range(100))

## Write the numbers in l to a file "out.txt"
## Read the numbers from out.txt into a new list, "l2"
## write to file read from file
## check that l == l2

# Reading

* Open book Chapter 13 on files: http://openbookproject.net/thinkcs/python/english3e/files.html

# Homework

* ZyBook Reading 11

